In [ ]:
import sys
import os

sys.path.append(os.path.dirname(os.getcwd()))

from MazeGenerationAlgorithms.LoopHuntAndKill import LoopHuntAndKillMaze
from MazeGenerationAlgorithms.LoopPrim import LoopPrimMaze

from MazeSolutionAlgorithms.AStar import AStarSolver
from MazeSolutionAlgorithms.BreadthFirstSearch import BFSSolver
from MazeSolutionAlgorithms.DepthFirstSearch import DFSSolver
from MazeSolutionAlgorithms.HybridSearch import HybridBFSDFS
from MazeSolutionAlgorithms.RandomWalk import RandomWalkSolver

import pandas as pd
import networkx as nx

In [ ]:
class DataExporter:

    def __init__(self, maze_objects):
        self.mazes = maze_objects
    
        self.solver_classes = {
            "A*": AStarSolver,
            "BFS": BFSSolver,
            "DFS": DFSSolver,
            "HybridBFSDFS": HybridBFSDFS,
            "RandomWalk": RandomWalkSolver
        }

        self.stats_df = self._get_visited_statistics()

    def _get_visited_statistics(self):
        stats = []

        for maze in self.mazes:

            solution_path = nx.shortest_path(maze.maze, maze.start, maze.end)
            solution_path_set = set(solution_path)
            solution_length = len(solution_path_set) - 1

            total_dead_ends = sum(1 for node in maze.maze.nodes
                              if maze.maze.degree(node) == 1 and node not in [maze.start, maze.end])

            total_intersections = sum(1 for node in maze.maze.nodes
                                  if maze.maze.degree(node) > 2)

            for solver_name, SolverClass in self.solver_classes.items():
                solver = SolverClass(maze)

                deadends = []
                intersections = []

                for node in solver.visited:
                    deg = maze.maze.degree(node)
                    if deg == 1 and node not in [maze.start, maze.end]:
                        deadends.append(node)
                    elif deg > 2:
                        intersections.append(node)

                clean_maze_name = solver.maze_name.replace(' Algorithm', '').strip()

                visited_count = len(solver.visited)
                visited_dead_end_count = len(deadends)
                visited_intersection_count = len(intersections)

                dead_end_coverage = round(visited_dead_end_count / total_dead_ends, 2) if total_dead_ends else 0.0
                intersection_coverage = round(visited_intersection_count / total_intersections, 2) if total_intersections else 0.0

                stats.append({
                    "Maze Type": clean_maze_name,
                    "Size": f"{maze.width}x{maze.height}",
                    "Loop Density": maze.loop_density,
                    "Solver": solver_name,
                    "Visited Nodes": visited_count,
                    "Solution Path Length": solution_length,
                    "Excess Steps": visited_count - solution_length,
                    "Exploration Ratio": (visited_count - solution_length) / solution_length,
                    "Visited Dead Ends": visited_dead_end_count,
                    "Visited Intersections": visited_intersection_count,
                    "Dead End Coverage": dead_end_coverage,
                    "Intersection Coverage": intersection_coverage
                })

        df = pd.DataFrame(stats)
        df.sort_values(["Maze Type", "Size"], inplace=True)

        return df

In [ ]:
width, height = 25, 25

maze_lst = []

for _ in range(100):
    maze_lst.extend([
        LoopHuntAndKillMaze(width, height),
        LoopPrimMaze(width, height),
        LoopHuntAndKillMaze(width, height, loop_density=0.1),
        LoopPrimMaze(width, height, loop_density=0.1),
        LoopHuntAndKillMaze(width, height, loop_density=0.25),
        LoopPrimMaze(width, height, loop_density=0.25),
    ])

data = DataExporter(maze_lst)

data.stats_df

In [ ]:
data.stats_df.to_csv('loop_solution.csv', index=False)